# Revised case normalization for USZ 2018

Notes:

- There are no BFS cases of the USZ for 2018. Therefore, no cases can be matched between the DtoD revised cases and the BFS USZ data.


In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')
from loguru import logger
import pandas as pd
from src.service.aimedic_grouper import group_batch_group_cases

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info=FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/USZ_2018-2019_20200730.xlsx'),
         'USZ',
         '2018',
         'Rückmeldungen_USZ_2018')

print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/USZ_2018-2019_20200730.xlsx', hospital_name_db='USZ', year='2018', sheet='Rückmeldungen_USZ_2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-10 14:32:17.118 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 169 cases for USZ 2018
2022-11-10 14:32:17.126 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:60 - TYPES:
case_id                  object
patient_id               object
kkik                     string
gender                   string
age_years                 int64
bfs_code                 string
duration_of_stay          int64
pflegetage neu           string
pccl                      int64
pccl neu                 string
old_pd                   string
primary_diagnosis        string
added_icds               string
removed_icds             string
added_chops              string
removed_chops            string
drg                      string
drg neu                  string
cw alt                   string
cw neu                   string
cw-änderung              string
kommentar                string
unnamed: 22              string
kodie

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,5771703,4424387,W,90,10,5771703,N390,N390,<NA>,3,L63D,[U804],[],[],[]
1,5773574,60210855,M,95,14,5773574,D042,D042,<NA>,3,J10Z,[B965],[],[],[]
2,5776284,10743014,W,83,5,5776284,I495,I495,<NA>,3,F12D,"[I5014, J4409]","[I5012, J4483]",[],[]
3,5793305,6214207,M,60,7,5793305,C343,C343,<NA>,3,E71B,"[J9600, J90]",[],[3491::20180117],[]
4,5816220,10776936,W,84,8,5816220,I214,I214,<NA>,3,F24C,[N184],[N183],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,5941848,10789030,W,76,31,5941848,H168,H168,1051,2,C60B,"[U8021, B482, B022, B957, B021]",[B952],[],[]
159,5984839,10388955,W,64,3,5984839,I480,I480,3354,2,F72B,"[U804, Z223, Z290]",[],[],[]
160,5990515,10387733,W,50,9,5990515,K565,K565,5453,2,G21B,"[C786, J91, C782, E440]",[],[],[]
161,5995298,10589554,M,60,33,5995298,K8520,K8520,6167,3,H62B,"[E440, F128, E871]",[E43],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

ValueError: There is no data for the hospital 'USZ' in 2018

In [ ]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [ ]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

In [ ]:
revisions_update

In [ ]:
update_db(revisions_update, diagnoses_update, procedures_update)